In [4]:
import numpy as np

In [1]:
pb = 1000.
pb_half = 975.
pt = 100.
pt_half = 125.

In [2]:
hPa = 0.01
g = 9.8066         # m/s2
kappa = 0.2856219  # poisson exponent = Rd/Cp_d
R = 287.05         # Gas constant for dry air at the surface; J/(K.kg)
p0 = 1.e5          # Reference pressure for potential temperature; Pa
alpha = -1./5.255877  # Density of dry air at 0C and 1000mb?
cp = 1004.7        # Specific heat at constant pressure for dry air; J/(K.kg)
r_earth = 6341.624  # I assume this is earth radius, but is that 6341 or 6371?
gamma = .0065      #The dry adiabatic lapse rate

alpha=-1./5.255877
beta=-6341.624

p00= 1e5 * hPa

In [20]:
def cal_zttheta(p):
    nlevels = p.size
    
    Z = np.zeros(nlevels)
    T = np.zeros(nlevels)
    theta = np.zeros(nlevels)


    P_tropopause = 226.32 #hPa
    tropopause_idx = np.where(p>=P_tropopause)[0][-1] + 1

    # calculate variables below the tropopause
    Z[:tropopause_idx] = (288.15/gamma) * ( 1. - (1013.25/p[:tropopause_idx])**alpha)
    T[:tropopause_idx] = 288.15 - 0.0065*Z[:tropopause_idx]
    theta[:tropopause_idx] = T[:tropopause_idx] * (p00/p[:tropopause_idx])**(R/cp)

    # calculate variables above the tropopause
    T[tropopause_idx:] = 216.65
    Z[tropopause_idx:] = (11.e3 + beta * np.log(p[tropopause_idx:] / 226.32))
    theta[:tropopause_idx] = T[:tropopause_idx] * (p00/p[:tropopause_idx])**(R/cp)
    
    return T, Z, theta

In [38]:
def diff(a, i=1):
    n = len(a)
    return a[i:]-a[:n-i]

In [5]:
p = np.linspace(pb,pt,19)
p_half = np.linspace(pb_half,pt_half,18)

In [62]:

nlevels = p.size
nlevels_half = p_half.size

T, Z, theta = cal_zttheta(p)
T_half, Z_half, theta_half = cal_zttheta(p_half)


dthetadp = np.zeros(nlevels_half)

dthetadp[0] = (theta[1] - theta[0])/(1.e2*(p[1]-p[0]))
dthetadp[1:] = diff(theta,i=2) / (1.e2 * diff(p, i=2))
S_half = - R * (T_half / theta_half) * dthetadp / (p_half*100)  # p[:-1]? or p_half

<ipython-input-62-a963e28bb212>:17: RuntimeWarning: divide by zero encountered in true_divide
  S_half = - R * (T_half / theta_half) * dthetadp / (p_half*100)
<ipython-input-62-a963e28bb212>:17: RuntimeWarning: invalid value encountered in multiply
  S_half = - R * (T_half / theta_half) * dthetadp / (p_half*100)


array([8.24603243e-07, 8.81627425e-07, 9.73416102e-07, 1.08095290e-06,
       1.20814109e-06, 1.36019089e-06, 1.54418791e-06, 1.76997491e-06,
       2.05156275e-06, 2.40947005e-06, 2.87476827e-06, 3.49643069e-06,
       4.35550610e-06, 5.59454757e-06, 7.48471082e-06,           -inf,
                 -inf,            nan])